# Neural Network Testing Framework
This notebook is intended to be a general framework for testing Neural Neworks on the EEG data set. While the initial iteration of this notebook is used for a simple Convolutional Neural Network (CNN), other networks using PyTorch can be implemented and assigned to the neural network variable. Many of the beginning cells are used to intialize for a Google Drive Colaboratory notebook GPU usability and can be ignored as necessary.

---



##Initialization
Google Drive access, PyTorch, etc.

In [0]:
!kill -9 -1

This section provides access to the user's Google drive.

In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


This section creates a drive that links to the user's Google drive.

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

This section installs PyTorch.

In [0]:
# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision

##Imports

Imports can be added as necessary.

In [0]:
import numpy as np
import h5py
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.cuda
from torch.utils.data import Dataset
from torch.autograd import Variable
from scipy import stats

##Classes

###EEGDataset
This class inherits the torch.utils.data.Dataset class to be used with the torch.utils.data.Dataloader class.

In [0]:
class EEGDataset(Dataset):
  """EEG dataset."""
  
  def __init__(self, x, y, transform=None):
    """
    Args:
      x (numpy array): Input data of shape 
                       num_trials x num_electrodes x num_time_bins.
      y (numpy array): Output data of shape num_trials x 1.
      transform (callable, optional): Optional transform to be applied.
    """
    self.x = x
    self.y = y
    self.transform = transform
    
  def __len__(self):
    return len(self.x)
  
  def __getitem__(self, idx):
    x_sample = torch.from_numpy(self.x[idx])
    y_sample = torch.IntTensor([int(self.y[idx])])
    
    if self.transform:
      pass #FIXME
    
    return x_sample, y_sample

###EEGMinimalContainer
This class holds a train and test EEGDataset. It processes the data into a (N, C, H, W) format in time.

In [0]:
class EEGMinimalContainer():
  """EEG container for training and testing datasets."""
  
  def __init__(self, data_dir, train_subject=None, test_subject=None, 
               remove_eog_channels=True, seed=42):
    """
    Args:
      data_dir (string): Path to all A0iT_slice.mat files for i in [1, 9].
      train_subject(int): Subject to train on. If None, train on all.
      test_subject(int): Subject to test on. If None, train on all except for
                         train_subject. Only used if train_subject is not None.
    """
    self.X_train = None
    self.y_train = None
    self.X_test = None
    self.y_test = None
    self.train_dataset = None
    self.test_dataset = None
    np.random.seed(seed)
    
    if train_subject is None:
      # Step 1: Append all of the input and output data together
      X = None
      y = None
      end = np.empty(9)
      for i in np.arange(9):
        A0iT = h5py.File(data_dir + ('/A0%dT_slice.mat' % (i+1)), 'r')
        X_temp = np.copy(A0iT['image'])
        y_temp = np.copy(A0iT['type'])
        y_temp = y_temp[0,0:X_temp.shape[0]:1]
        y_temp = np.asarray(y_temp, dtype=np.int32)
        X = X_temp if X is None else np.append(X, X_temp, axis=0)
        y = y_temp if y is None else np.append(y, y_temp, axis=0)
        end[i] = X_temp.shape[0] if i == 0 else X_temp.shape[0] + end[i-1]
      X = np.expand_dims(X, axis=1)
      y -= 769
      # Step 2: Remove the EOG
      if remove_eog_channels:
        X = X[:, :, 0:22, :] 
      # Step 3: Remove NaN trials
      remove_list = []
      for i in range(len(X)):
        if np.isnan(X[i]).any():
          remove_list.append(i)
      for trial_row in remove_list:
        end[end > trial_row] -= 1
      X = np.delete(X, remove_list, axis=0)
      y = np.delete(y, remove_list, axis=0)
      # Step 4: Generate an train/test split
      remove_list = []
      self.X_test = {}
      self.y_test = {}
      self.test_dataset = {}
      sloc = 0
      for i, eloc in enumerate(end, 1):
        t_list = np.random.choice(np.arange(sloc, eloc), 50, replace=False)
        t_list = t_list.astype(int)
        self.X_test[str(i)] = X[t_list, :, :, :]
        self.y_test[str(i)] = y[t_list]
        self.test_dataset[str(i)] = EEGDataset(X[t_list, :, :, :], y[t_list])
        remove_list = remove_list + t_list.tolist()
        sloc = eloc
      self.X_train = np.delete(X, remove_list, axis=0)
      self.y_train = np.delete(y, remove_list, axis=0)
      self.train_dataset = EEGDataset(self.X_train, self.y_train)
      
      print('EEGContainer X_train: ' + str(self.X_train.shape))
      print('EEGContainer y_train: ' + str(self.y_train.shape))
      for i in range(1, 10):
        print(('EEGContainer X_test%d: ' %i) + str(self.X_test[str(i)].shape))
        print(('EEGContainer y_test%d: ' %i) + str(self.y_test[str(i)].shape))
    
    else:
      pass #FIXME

####EEGCroppedContainer



In [0]:
class EEGCroppedContainer():
  """EEG container for training and testing datasets."""
  
  def __init__(self, data_dir, crop_size, train_subject=None, test_subject=None, 
               remove_eog_channels=True, seed=42):
    """
    Args:
      data_dir (string): Path to all A0iT_slice.mat files for i in [1, 9].
      train_subject(int): Subject to train on. If None, train on all.
      test_subject(int): Subject to test on. If None, train on all except for
                         train_subject. Only used if train_subject is not None.
    """
    self.X_train = None
    self.y_train = None
    self.X_test = None
    self.y_test = None
    self.train_dataset = None
    self.test_dataset = None
    np.random.seed(seed)
    
    if train_subject is None:
      # Step 1: Append all of the input and output data together
      X = None
      y = None
      end = np.empty(9)
      for i in np.arange(9):
        A0iT = h5py.File(data_dir + ('/A0%dT_slice.mat' % (i+1)), 'r')
        X_temp = np.copy(A0iT['image'])
        y_temp = np.copy(A0iT['type'])
        y_temp = y_temp[0,0:X_temp.shape[0]:1]
        y_temp = np.asarray(y_temp, dtype=np.int32)
        X = X_temp if X is None else np.append(X, X_temp, axis=0)
        y = y_temp if y is None else np.append(y, y_temp, axis=0)
        end[i] = X_temp.shape[0] if i == 0 else X_temp.shape[0] + end[i-1]
      X = np.expand_dims(X, axis=1)
      y -= 769
      # Step 2: Remove the EOG
      if remove_eog_channels:
        X = X[:, :, 0:22, :] 
      # Step 3: Remove NaN trials
      remove_list = []
      for i in range(len(X)):
        if np.isnan(X[i]).any():
          remove_list.append(i)
      for trial_row in remove_list:
        end[end > trial_row] -= 1
      X = np.delete(X, remove_list, axis=0)
      y = np.delete(y, remove_list, axis=0)
      # Step 4: Generate an test split
      remove_list = []
      self.X_test = {}
      self.y_test = {}
      self.test_dataset = {}
      sloc = 0
      for i, eloc in enumerate(end, 1):
        t_list = np.random.choice(np.arange(sloc, eloc), 50, replace=False)
        t_list = t_list.astype(int)
        self.X_test[str(i)] = X[t_list, :, :, :]
        self.y_test[str(i)] = y[t_list]
        self.test_dataset[str(i)] = EEGDataset(X[t_list, :, :, :], y[t_list])
        remove_list = remove_list + t_list.tolist()
        sloc = eloc
      # Step 5: Go through and create a cropped train split
      X_train = np.delete(X, remove_list, axis=0)
      y_train = np.delete(y, remove_list, axis=0)
      self.X_train = None
      for i in range(1000 - crop_size):
        X_train_temp = X_train[:, :, :, i:(i+crop_size)]
        if self.X_train is None:
          self.X_train = X_train_temp
        else:
          self.X_train = np.concatenate((self.X_train, X_train_temp), 0)
      self.y_train = np.tile(y_train, 1000 - crop_size)
      self.train_dataset = EEGDataset(self.X_train, self.y_train)
      
      print('EEGContainer X_train: ' + str(self.X_train.shape))
      print('EEGContainer y_train: ' + str(self.y_train.shape))
      for i in range(1, 10):
        print(('EEGContainer X_test%d: ' %i) + str(self.X_test[str(i)].shape))
        print(('EEGContainer y_test%d: ' %i) + str(self.y_test[str(i)].shape))
    
    else:
      pass #FIXME

###Fully-Connected Neural Network

####FNN1

Extremely Vanilla FNN

In [0]:
class FNN1(nn.Module):
  def __init__(self):
    super(FNN1, self).__init__()
    self.input_size = 1 * 22 * 1000
    self.fc1 = nn.Linear(self.input_size, 1000)
    self.fc2 = nn.Linear(1000, 500)
    self.fc3 = nn.Linear(500, 200)
    self.fc4 = nn.Linear(200, 100)
    self.fc5 = nn.Linear(100, 4)
    
  def forward(self, x):
    x = x.view(-1, self.input_size)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.relu(self.fc3(x))
    x = F.relu(self.fc4(x))
    x = F.relu(self.fc5(x))
    return x

###Convolutional Neural Network

####CNN1 (CNN1)
Extremely Vanilla CNN. Note the 3x3 convolutions do not take advantage of the temporal information.

conv+relu - conv+relu - pool - fc+relu - fc+relu - fc

In [0]:
class CNN1(nn.Module):
  def __init__(self):
    super(CNN1, self).__init__()
    self.conv1 = nn.Conv2d(1, 4, 3, stride=1, padding=1)
    self.conv2 = nn.Conv2d(4, 8, 3, stride=1, padding=1)
    self.pool = nn.MaxPool2d(2, 2)
    self.fc1 = nn.Linear(8 * 11 * 500, 120)
    self.fc2 = nn.Linear(120, 80)
    self.fc3 = nn.Linear(80, 4)
  
  def forward(self, x):
    # x is 1 x 22 x 1000
    x = self.pool(F.relu(self.conv2(F.relu(self.conv1(x)))))
    # x is 8 x 11 x  500
    x = x.view(-1, 8 * 11 * 500)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x    

####CNN2

CNN now consider temporal dependencies. Does marginally better than first CNN; we may need to add BatchNorm / Dropout / Weight Normalization.

[conv - conv - pool] x 2 - [conv - pool] x 3 - [fc] x 3

In [0]:
class CNN2(nn.Module):
  def __init__(self):
    super(CNN2, self).__init__()
    self.conv1 = nn.Conv2d( 1, 16, (1, 11), stride=(1, 1), padding=0)
    self.conv2 = nn.Conv2d(16, 16, (22, 1), stride=(1, 1), padding=0)
    self.conv3 = nn.Conv2d( 1, 16, (1, 11), stride=(1, 1), padding=0)
    self.conv4 = nn.Conv2d(16, 16, (16, 1), stride=(1, 1), padding=0)
    self.conv5 = nn.Conv2d( 1, 16, 3, stride=1, padding=1)
    self.conv6 = nn.Conv2d(16, 32, 3, stride=1, padding=1)
    self.conv7 = nn.Conv2d(32, 64, 3, stride=1, padding=1)
    self.fc_1 = nn.Linear(64 * 2 * 10, 200)
    self.fc_2 = nn.Linear(200, 100)
    self.fc_3 = nn.Linear(100, 4)
  
  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = F.relu(self.conv2(x))
    x = x.permute(0, 2, 1, 3)
    x = F.max_pool2d(x, (1, 3), (1, 3))
    x = F.relu(self.conv3(x))
    x = F.relu(self.conv4(x))
    x = x.permute(0, 2, 1, 3)
    x = F.max_pool2d(x, (1, 4), (1, 4))
    x = F.max_pool2d(F.relu(self.conv5(x)), 2, 2)
    x = F.max_pool2d(F.relu(self.conv6(x)), 2, 2)
    x = F.max_pool2d(F.relu(self.conv7(x)), 2, 2)
    x = x.view(-1, 64 * 2 * 10)
    x = F.relu(self.fc_1(x))
    x = F.relu(self.fc_2(x))
    x = self.fc_3(x)
    return x

####CNN3 (CNN2)

An attempt to simplify CNN2. Did not come to expectation, due to less layers.

[conv - conv - pool] x 1 - [conv - pool] x 1 - [fc] x 3

In [0]:
class CNN3(nn.Module):
  def __init__(self):
    super(CNN3, self).__init__()
    self.conv1 = nn.Conv2d( 1, 16, (1, 11), stride=1, padding=0)
    self.conv2 = nn.Conv2d(16, 16, (22, 1), stride=1, padding=0)
    self.pool_1 = nn.MaxPool2d((1, 3), (1, 3))
    self.conv3 = nn.Conv2d(1, 16, 3, stride=1, padding=1)
    self.pool_2 = nn.MaxPool2d(2, 2)
    self.fc_1 = nn.Linear(16 * 8 * 165, 120)
    self.fc_2 = nn.Linear(120, 80)
    self.fc_3 = nn.Linear(80, 4)
  
  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = F.relu(self.conv2(x))
    x = self.pool_1(x.permute(0, 2, 1, 3))
    x = self.pool_2(F.relu(self.conv3(x)))
    x = x.view(-1, 16 * 8 * 165)
    x = F.relu(self.fc_1(x))
    x = F.relu(self.fc_2(x))
    x = self.fc_3(x)
    return x

####CNN4 (CNN3)

Let's consider long time dependencies.



In [0]:
class CNN4(nn.Module):
  def __init__(self):
    super(CNN4, self).__init__()
    self.conv1 = nn.Conv2d( 1, 16, (1, 11), stride=1, padding=0)
    self.conv2 = nn.Conv2d(16, 16, (22, 1), stride=1, padding=0)
    self.conv3 = nn.Conv2d( 1, 16, (1, 75), stride=1, padding=0)
    self.conv4 = nn.Conv2d(16, 32, 3, stride=1, padding=1)
    self.conv5 = nn.Conv2d(32, 64, 3, stride=1, padding=1)
    self.fc1 = nn.Linear(64 * 2 * 32, 120)
    self.fc2 = nn.Linear(120, 80)
    self.fc3 = nn.Linear(80, 4)
  
  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = F.relu(self.conv2(x))
    x = x.permute(0, 2, 1, 3)
    x = F.max_pool2d(x, (1, 3), (1, 3))
    x = F.max_pool2d(F.relu(self.conv3(x)), 2, 2)
    x = F.max_pool2d(F.relu(self.conv4(x)), 2, 2)
    x = F.max_pool2d(F.relu(self.conv5(x)), 2, 2)
    x = x.view(-1, 64 * 2 * 32)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

####CNN5

CNN4 with BatchNorm and Dropout...

In [0]:
class CNN5(nn.Module):
  def __init__(self):
    super(CNN5, self).__init__()
    self.conv1 = nn.Conv2d( 1, 16, (1, 11), stride=1, padding=0)
    self.bnc1 = nn.BatchNorm2d(16)
    self.conv2 = nn.Conv2d(16, 16, (22, 1), stride=1, padding=0)
    self.bnc2 = nn.BatchNorm2d(16)
    self.conv3 = nn.Conv2d( 1, 16, (1, 75), stride=1, padding=0)
    self.bnc3 = nn.BatchNorm2d(16)
    self.conv4 = nn.Conv2d(16, 32, 3, stride=1, padding=1)
    self.bnc4 = nn.BatchNorm2d(32)
    self.conv5 = nn.Conv2d(32, 64, 3, stride=1, padding=1)
    self.bnc5 = nn.BatchNorm2d(64)
    self.fc1 = nn.Linear(64 * 2 * 32, 120)
    self.bnf1 = nn.BatchNorm1d(120)
    self.fc2 = nn.Linear(120, 80)
    self.bnf2 = nn.BatchNorm1d(80)
    self.fc3 = nn.Linear(80, 4)
  
  def forward(self, x):
    x = F.dropout(self.bnc1(F.relu(self.conv1(x))), p=0.2)
    x = F.dropout(self.bnc2(F.relu(self.conv2(x))))
    x = x.permute(0, 2, 1, 3)
    x = F.max_pool2d(x, (1, 3), (1, 3))
    x = F.dropout(self.bnc3(F.relu(self.conv3(x))))
    x = F.max_pool2d(x, 2, 2)
    x = F.dropout(self.bnc4(F.relu(self.conv4(x))))
    x = F.max_pool2d(x, 2, 2)
    x = F.dropout(self.bnc5(F.relu(self.conv5(x))))
    x = F.max_pool2d(x, 2, 2)
    x = x.view(-1, 64 * 2 * 32)
    x = F.dropout(self.bnf1(F.relu(self.fc1(x))))
    x = F.dropout(self.bnf2(F.relu(self.fc2(x))))
    x = self.fc3(x)
    return x

####CNN6

CNN2 with BatchNorm and Dropout...

BatchNorm is a disaster, dropping the validation accuracy to 0.34

In [0]:
class CNN6(nn.Module):
  def __init__(self):
    super(CNN6, self).__init__()
    self.conv1 = nn.Conv2d( 1, 16, (1, 11), stride=(1, 1), padding=0)
    self.bnc1 = nn.BatchNorm2d(16)
    self.conv2 = nn.Conv2d(16, 16, (22, 1), stride=(1, 1), padding=0)
    self.bnc2 = nn.BatchNorm2d(16)
    self.conv3 = nn.Conv2d( 1, 16, (1, 11), stride=(1, 1), padding=0)
    self.bnc3 = nn.BatchNorm2d(16)
    self.conv4 = nn.Conv2d(16, 16, (16, 1), stride=(1, 1), padding=0)
    self.bnc4 = nn.BatchNorm2d(16)
    self.conv5 = nn.Conv2d( 1, 16, 3, stride=1, padding=1)
    self.bnc5 = nn.BatchNorm2d(16)
    self.conv6 = nn.Conv2d(16, 32, 3, stride=1, padding=1)
    self.bnc6 = nn.BatchNorm2d(32)
    self.conv7 = nn.Conv2d(32, 64, 3, stride=1, padding=1)
    self.bnc7 = nn.BatchNorm2d(64)
    self.fc1 = nn.Linear(64 * 2 * 10, 200)
    self.bnf1 = nn.BatchNorm1d(200)
    self.fc2 = nn.Linear(200, 100)
    self.bnf2 = nn.BatchNorm1d(100)
    self.fc3 = nn.Linear(100, 4)
  
  def forward(self, x):
    x = F.dropout(self.bnc1(F.relu(self.conv1(x))), p=0.2)
    x = F.dropout(self.bnc2(F.relu(self.conv2(x))))
    x = x.permute(0, 2, 1, 3)
    x = F.max_pool2d(x, (1, 3), (1, 3))
    x = F.dropout(self.bnc3(F.relu(self.conv3(x))))
    x = F.dropout(self.bnc4(F.relu(self.conv4(x))))
    x = x.permute(0, 2, 1, 3)
    x = F.max_pool2d(x, (1, 4), (1, 4))
    x = F.dropout(self.bnc5(F.relu(self.conv5(x))))
    x = F.max_pool2d(x, 2, 2)
    x = F.dropout(self.bnc6(F.relu(self.conv6(x))))
    x = F.max_pool2d(x, 2, 2)
    x = F.dropout(self.bnc7(F.relu(self.conv7(x))))
    x = F.max_pool2d(x, 2, 2)
    x = x.view(-1, 64 * 2 * 10)
    x = F.dropout(self.bnf1(F.relu(self.fc1(x))))
    x = F.dropout(self.bnf2(F.relu(self.fc2(x))))
    x = self.fc3(x)
    return x

####CNN7 (CNN4)

CNN2 with only Dropout? Best validation accuracy: 0.50

In [0]:
class CNN7(nn.Module):
  def __init__(self):
    super(CNN7, self).__init__()
    self.conv1 = nn.Conv2d( 1, 16, (1, 11), stride=(1, 1), padding=0)
    self.conv2 = nn.Conv2d(16, 16, (22, 1), stride=(1, 1), padding=0)
    self.conv3 = nn.Conv2d( 1, 16, (1, 11), stride=(1, 1), padding=0)
    self.conv4 = nn.Conv2d(16, 16, (16, 1), stride=(1, 1), padding=0)
    self.conv5 = nn.Conv2d( 1, 16, 3, stride=1, padding=1)
    self.conv6 = nn.Conv2d(16, 32, 3, stride=1, padding=1)
    self.conv7 = nn.Conv2d(32, 64, 3, stride=1, padding=1)
    self.fc1 = nn.Linear(64 * 2 * 10, 200)
    self.fc2 = nn.Linear(200, 100)
    self.fc3 = nn.Linear(100, 4)
  
  def forward(self, x):
    dropval = 0.7
    x = F.dropout2d(F.relu(self.conv1(x)), p=0.2)
    x = F.dropout2d(F.relu(self.conv2(x)), p=dropval)
    x = x.permute(0, 2, 1, 3)
    x = F.max_pool2d(x, (1, 3), (1, 3))
    x = F.dropout2d(F.relu(self.conv3(x)), p=dropval)
    x = F.dropout2d(F.relu(self.conv4(x)), p=dropval)
    x = x.permute(0, 2, 1, 3)
    x = F.max_pool2d(x, (1, 4), (1, 4))
    x = F.dropout2d(F.relu(self.conv5(x)), p=dropval)
    x = F.max_pool2d(x, 2, 2)
    x = F.dropout2d(F.relu(self.conv6(x)), p=dropval)
    x = F.max_pool2d(x, 2, 2)
    x = F.dropout2d(F.relu(self.conv7(x)), p=dropval)
    x = F.max_pool2d(x, 2, 2)
    x = x.view(-1, 64 * 2 * 10)
    x = F.dropout(F.relu(self.fc1(x)), p=dropval)
    x = F.dropout(F.relu(self.fc2(x)), p=dropval)
    x = self.fc3(x)
    return x

####CNN8 (CNN5)

Brand new CNN. Attempts to use the average pool. Similar validation accuracy: 0.48.

In [0]:
class CNN8(nn.Module):
  def __init__(self):
    super(CNN8, self).__init__()
    self.conv1 = nn.Conv2d( 1, 20, ( 1, 8))
    self.conv2 = nn.Conv2d(20, 20, (22, 1))
    self.conv3 = nn.Conv2d( 1,  8, ( 1, 4))
    self.conv4 = nn.Conv2d( 8, 16, 3, padding=1)
    self.conv5 = nn.Conv2d(16, 32, 5, padding=2)
    self.conv6 = nn.Conv2d(32, 32, 3, padding=1)
    self.conv7 = nn.Conv2d(32, 64, 5, padding=2)
    self.conv8 = nn.Conv2d(64, 64, 3, padding=1)
    self.fc1 = nn.Linear(64 * 5 * 41, 200)
    self.fc2 = nn.Linear(200, 100)
    self.fc3 = nn.Linear(100, 100)
    self.fc4 = nn.Linear(100, 4)
  
  def forward(self, x):
    dropval = 0.75
    x = F.dropout2d(F.relu(self.conv1(x)), p=0.2)
    x = F.dropout2d(F.relu(self.conv2(x)), p=dropval)
    x = x.permute(0, 2, 1, 3)
    x = F.avg_pool2d(x, (1, 3), (1, 3))
    x = F.dropout2d(F.relu(self.conv3(x)), p=dropval)
    x = F.dropout2d(F.relu(self.conv4(x)), p=dropval)
    x = F.avg_pool2d(x, (1, 2), (1, 2))
    x = F.dropout2d(F.relu(self.conv5(x)), p=dropval)
    x = F.dropout2d(F.relu(self.conv6(x)), p=dropval)
    x = F.avg_pool2d(x, 2, 2)
    x = F.dropout2d(F.relu(self.conv7(x)), p=dropval)
    x = F.dropout2d(F.relu(self.conv8(x)), p=dropval)
    x = F.avg_pool2d(x, 2, 2)
    x = x.view(-1, 64 * 5 * 41)
    x = F.dropout(F.relu(self.fc1(x)), p=dropval)
    x = F.dropout(F.relu(self.fc2(x)), p=dropval)
    x = F.dropout(F.relu(self.fc3(x)), p=dropval)
    x = self.fc4(x)
    return x

####CNN9 (CNN6)

Introduces a feature called "Sprinkle Shuffle", a method similar to the Google Inception method. 

Ideas: "Nerf" the depth of the network. 

In [0]:
# N x 1 x 22 x 1000

class CNN9(nn.Module):
  def __init__(self):
    super(CNN9, self).__init__()
    self.sprinkle1 = nn.Conv2d(1,  7, ( 1, 75), padding=(0, 37))
    self.sprinkle2 = nn.Conv2d(1,  7, ( 1, 43), padding=(0, 21))
    self.sprinkle3 = nn.Conv2d(1,  7, ( 1, 25), padding=(0, 12))
    self.sprinkle4 = nn.Conv2d(1,  7, ( 1, 11), padding=(0,  5))
    self.sprinkle5 = nn.Conv2d(1, 22, (22,  1))
    self.sprinkle6 = nn.Conv2d(1, 22, (22,  5), padding=(0,  2))
    self.sprinkle7 = nn.Conv2d(1, 22, (22,  9), padding=(0,  4))
    self.sprinkle8 = nn.Conv2d(1, 22, (22, 11), padding=(0,  5))
    self.inception1_1 = nn.Conv2d(32, 8, 1, padding=0)
    self.inception1_3 = nn.Conv2d( 8, 8, 3, padding=1)
    self.inception1_5 = nn.Conv2d( 8, 8, 5, padding=2)
    self.reduce1 = nn.Conv2d(32, 32, (5, 4), stride=(1, 2), padding=(0, 1))
    self.conv1 = nn.Conv2d( 32,  64, 3, padding=1)
    self.conv2 = nn.Conv2d( 64,  64, 3, padding=1)
    self.conv3 = nn.Conv2d( 64, 128, 3, padding=1)
    self.conv4 = nn.Conv2d(128, 128, 3, padding=1)
    self.fc = nn.Linear(128 * 10 * 125, 4)
  
  def forward(self, x):
    dropval = 0.5
    xold = x
    # Collect all the sprinkles
    s1 = F.relu(self.sprinkle1(x))
    s2 = F.relu(self.sprinkle2(x))
    s3 = F.relu(self.sprinkle3(x))
    s4 = F.relu(self.sprinkle4(x))
    s5 = (F.relu(self.sprinkle5(x))).permute(0, 2, 1, 3)
    s6 = (F.relu(self.sprinkle6(x))).permute(0, 2, 1, 3)
    s7 = (F.relu(self.sprinkle7(x))).permute(0, 2, 1, 3)
    s8 = (F.relu(self.sprinkle8(x))).permute(0, 2, 1, 3)
    # Concatenate the sprinkles together
    xp = torch.cat((s1, s2, s3, s4, s5, s6, s7, s8), 1)
    # Create a "new" x
    x = -x[:, :, np.random.permutation(22), :]
    s1 = F.relu(self.sprinkle1(x))
    s2 = F.relu(self.sprinkle2(x))
    s3 = F.relu(self.sprinkle3(x))
    s4 = F.relu(self.sprinkle4(x))
    s5 = (F.relu(self.sprinkle5(x))).permute(0, 2, 1, 3)
    s6 = (F.relu(self.sprinkle6(x))).permute(0, 2, 1, 3)
    s7 = (F.relu(self.sprinkle7(x))).permute(0, 2, 1, 3)
    s8 = (F.relu(self.sprinkle8(x))).permute(0, 2, 1, 3)
    # Concatenate the sprinkles together
    xn = torch.cat((s1, s2, s3, s4, s5, s6, s7, s8), 1)
    x = torch.cat((xp + xold, xn + xold), 2) # Residual?
    x = F.dropout2d(x, p=0.2)
    xold = x
    # Inception Round 1
    i_temp = self.inception1_1(x)
    i1 = F.relu(i_temp)
    i2 = F.relu(self.inception1_3(i_temp))
    i3 = F.relu(self.inception1_5(i_temp))
    i4 = self.inception1_1(F.max_pool2d(x, 3, stride=1, padding=1))
    x = torch.cat((i1, i2, i3, i4), 1) + xold
    x = F.dropout2d(x, p=dropval)
    # Reduce Dimensions
    x = F.relu(self.reduce1(x))
    x = F.dropout(x, p=dropval)
    x = F.relu(self.conv1(x))
    x = F.relu(self.conv2(x))
    x = F.max_pool2d(x, 2)
    x = F.dropout(x, p=dropval)
    x = F.relu(self.conv3(x))
    x = F.relu(self.conv4(x))
    x = F.max_pool2d(x, 2)
    x = F.dropout(x, p=dropval)
    x = x.view(-1, 128 * 10 * 125)
    x = self.fc(x)
    return x

####CNN10

In [0]:
# N x 1 x 22 x 250

class CNN10(nn.Module):
  def __init__(self, time_batch=400):
    super(CNN10, self).__init__()
    self.sprinkle1 = nn.Conv2d(1,  7, ( 1, 75), padding=(0, 37))
    self.sprinkle2 = nn.Conv2d(1,  7, ( 1, 43), padding=(0, 21))
    self.sprinkle3 = nn.Conv2d(1,  7, ( 1, 25), padding=(0, 12))
    self.sprinkle4 = nn.Conv2d(1,  7, ( 1, 11), padding=(0,  5))
    self.sprinkle5 = nn.Conv2d(1, 22, (22,  1))
    self.sprinkle6 = nn.Conv2d(1, 22, (22,  5), padding=(0,  2))
    self.sprinkle7 = nn.Conv2d(1, 22, (22,  9), padding=(0,  4))
    self.sprinkle8 = nn.Conv2d(1, 22, (22, 11), padding=(0,  5))
    self.inception1_1 = nn.Conv2d(32, 8, 1, padding=0)
    self.inception1_3 = nn.Conv2d( 8, 8, 3, padding=1)
    self.inception1_5 = nn.Conv2d( 8, 8, 5, padding=2)
    self.inception2_1 = nn.Conv2d(32, 4, 1, padding=0)
    self.inception2_3 = nn.Conv2d( 4, 4, 3, padding=1)
    self.inception2_5 = nn.Conv2d( 4, 4, 5, padding=2)
    self.conv1 = nn.Conv2d(16, 32, 3, padding=1)
    self.conv2 = nn.Conv2d(32, 32, 3, padding=1)
    self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
    self.conv4 = nn.Conv2d(64, 64, 3, padding=1)
    if   time_batch == 250:
      self.fc = nn.Linear(64 * 11 * 31, 4) # For T=250
    elif time_batch == 400:
      self.fc = nn.Linear(64 * 11 * 49, 4) # For T=400
  
  def forward(self, x):
    time_batch = x.size(3)
    dropval = 0.5
    xold = x
    # Collect all the sprinkles
    s1 = F.relu(self.sprinkle1(x))
    s2 = F.relu(self.sprinkle2(x))
    s3 = F.relu(self.sprinkle3(x))
    s4 = F.relu(self.sprinkle4(x))
    s5 = (F.relu(self.sprinkle5(x))).permute(0, 2, 1, 3)
    s6 = (F.relu(self.sprinkle6(x))).permute(0, 2, 1, 3)
    s7 = (F.relu(self.sprinkle7(x))).permute(0, 2, 1, 3)
    s8 = (F.relu(self.sprinkle8(x))).permute(0, 2, 1, 3)
    # Concatenate the sprinkles together
    xp = torch.cat((s1, s2, s3, s4, s5, s6, s7, s8), 1)
    # Create a "new" x
    x = -x[:, :, np.random.permutation(22), :]
    s1 = F.relu(self.sprinkle1(x))
    s2 = F.relu(self.sprinkle2(x))
    s3 = F.relu(self.sprinkle3(x))
    s4 = F.relu(self.sprinkle4(x))
    s5 = (F.relu(self.sprinkle5(x))).permute(0, 2, 1, 3)
    s6 = (F.relu(self.sprinkle6(x))).permute(0, 2, 1, 3)
    s7 = (F.relu(self.sprinkle7(x))).permute(0, 2, 1, 3)
    s8 = (F.relu(self.sprinkle8(x))).permute(0, 2, 1, 3)
    # Concatenate the sprinkles together
    xn = torch.cat((s1, s2, s3, s4, s5, s6, s7, s8), 1)
    x = torch.cat((xp + xold, xn + xold), 2) # Residual?
    x = F.dropout2d(x, p=0.2)
    xold = x
    # Inception Round 1
    i_temp = self.inception1_1(x)
    i1 = F.relu(i_temp)
    i2 = F.relu(self.inception1_3(i_temp))
    i3 = F.relu(self.inception1_5(i_temp))
    i4 = self.inception1_1(F.max_pool2d(x, 3, stride=1, padding=1))
    x = torch.cat((i1, i2, i3, i4), 1) + xold
    x = F.dropout2d(x, p=dropval)
    xold = x
    # Inception Round 2
    i_temp = self.inception2_1(x)
    i1 = F.relu(i_temp)
    i2 = F.relu(self.inception2_3(i_temp))
    i3 = F.relu(self.inception2_5(i_temp))
    i4 = self.inception2_1(F.max_pool2d(x, 3, stride=1, padding=1))
    x = torch.cat((i1, i2, i3, i4), 1)
    x = F.dropout2d(x, p=dropval)
    # Reduce dimensions
    if   time_batch == 250:
      x = F.max_pool2d(x, (1, 4), stride=(1, 2))  # For T=250
    elif time_batch == 400:
      x = F.max_pool2d(x, (1, 10), stride=(1, 2)) # For T=400
    x = F.dropout(x, p=dropval)
    # Run through a standard neural net
    x = F.relu(self.conv1(x))
    x = F.relu(self.conv2(x))
    x = F.max_pool2d(x, 2)
    x = F.dropout(x, p=dropval)
    x = F.relu(self.conv3(x))
    x = F.relu(self.conv4(x))
    x = F.max_pool2d(x, 2)
    x = F.dropout(x, p=dropval)
    if   time_batch == 250:
      x = x.view(-1, 64 * 11 * 31) # For T=250
    elif time_batch == 400:
      x = x.view(-1, 64 * 11 * 49) # For T=400
    x = self.fc(x)
    return x

####CNN11

Shallow ConvNet for raw EEG signals (Schirrmeister)

Testing Accuracy: [0.56 0.38 0.72 0.62 0.38 0.48 0.54 0.68 0.7 ]

Testing Accuracy Average: 0.56222

learning_rate = 1e-4

Ran for 200 epochs

In [0]:
# N x 1 x 22 x 534
class CNN11(nn.Module):
  def __init__(self):
    super(CNN11, self).__init__()
    self.conv1 = nn.Conv2d( 1, 40, (1, 25))
    #nn.init.xavier_uniform(self.conv1.weight)
    self.bnc1 = nn.BatchNorm2d(40)
    self.conv2 = nn.Conv2d(40, 40, (22, 1))
    #nn.init.xavier_uniform(self.conv2.weight)
    self.bnc2 = nn.BatchNorm2d(40)
    self.fc = nn.Linear(30 * 40, 4)
    #nn.init.xavier_uniform(self.fc.weight)
  
  def forward(self, x):
    x = F.elu(self.bnc1(self.conv1(x)))
    x = F.elu(self.bnc2(self.conv2(x)))
    x = F.dropout2d(x, p=0.5)
    x = F.max_pool2d(x.permute(0, 2, 1, 3), (1, 75), stride=(1, 15))
    x = x.view(-1, 30 * 40)
    x = self.fc(x)
    return x

####CNN12

Deep ConvNet for raw EEG signals (Schirrmeister)

Validation Accuracy: 0.56444

learning_rate = 1e-4

In [0]:
# N x 1 x 22 x 522
class CNN12(nn.Module):
  def __init__(self, time_batch=400):
    super(CNN12, self).__init__()
    self.conv1 = nn.Conv2d( 1,  25, (  1, 10))
    self.bnc1 = nn.BatchNorm2d(25)
    self.conv2 = nn.Conv2d(25,  25, ( 22,  1))
    self.bnc2 = nn.BatchNorm2d(25)
    self.conv3 = nn.Conv2d( 1,  50, ( 25, 10))
    self.bnc3 = nn.BatchNorm2d(50)
    self.conv4 = nn.Conv2d( 1, 100, ( 50, 10))
    self.bnc4 = nn.BatchNorm2d(100)
    self.conv5 = nn.Conv2d( 1, 200, (100, 10))
    self.bnc5 = nn.BatchNorm2d(200)
    self.fc = nn.Linear(200 * 2, 4)
  
  def forward(self, x):
    x = F.elu(self.bnc1(self.conv1(x)))
    x = F.elu(self.bnc2(self.conv2(x)))
    x = F.dropout2d(x, p=0.5)
    x = F.max_pool2d(x.permute(0, 2, 1, 3), (1, 3))
    x = F.elu(self.bnc3(self.conv3(x)))
    x = F.dropout2d(x, p=0.5)
    x = F.max_pool2d(x.permute(0, 2, 1, 3), (1, 3))
    x = F.elu(self.bnc4(self.conv4(x)))
    x = F.dropout2d(x, p=0.5)
    x = F.max_pool2d(x.permute(0, 2, 1, 3), (1, 3))
    x = F.elu(self.bnc5(self.conv5(x)))
    x = F.dropout2d(x, p=0.5)
    x = F.max_pool2d(x.permute(0, 2, 1, 3), (1, 3))
    x = x.view(-1, 200 * 2)
    x = self.fc(x)
    return x

####CNN13

EEGNet

Validation Accuracy: 0.56000 but varies a lot



In [0]:
# N x 1 x 22 x 512
class CNN13(nn.Module):
  def __init__(self, time_batch=400):
    super(CNN13, self).__init__()
    self.conv1 = nn.Conv2d(1, 16, (22,  1))
    self.bnc1 = nn.BatchNorm2d(16)
    self.padding1 = nn.ZeroPad2d((16, 17, 0, 1))
    self.conv2 = nn.Conv2d(1,  4, ( 2, 32))
    self.bnc2 = nn.BatchNorm2d(4)
    self.padding2 = nn.ZeroPad2d(( 2,  1, 4, 3))
    self.conv3 = nn.Conv2d(4,  4, ( 8,  4))
    self.bnc3 = nn.BatchNorm2d(4)
    self.fc = nn.Linear(4 * 4 * 32, 4)
  
  def forward(self, x):
    x = F.elu(self.conv1(x))
    x = self.bnc1(x)
    x = F.dropout(x, p=0.25)
    x = x.permute(0, 2, 1, 3)
    x = self.padding1(x)
    x = F.elu(self.conv2(x))
    x = self.bnc2(x)
    x = F.dropout(x, p=0.25)
    x = F.max_pool2d(x, (2, 4))
    x = self.padding2(x)
    x = F.elu(self.conv3(x))
    x = self.bnc2(x)
    x = F.dropout(x, p=0.25)
    x = F.max_pool2d(x, (2, 4))
    x = x.view(-1, 4 * 4 * 32)
    x = self.fc(x)
    return x

####CNN14

CNN11 with Google Inception Input

Testing Accuracy: [0.64 0.46 0.64 0.44 0.42 0.52 0.58 0.62 0.68]

Testing Accuracy Average: 0.55556

In [0]:
# N x 1 x 22 x 510
class CNN14(nn.Module):
  def __init__(self):
    super(CNN14, self).__init__()
    self.conv1_1 = nn.Conv2d(1, 10, ( 1, 75), padding=(0, 37))
    self.conv1_2 = nn.Conv2d(1, 10, ( 1, 43), padding=(0, 21))
    self.conv1_3 = nn.Conv2d(1, 10, ( 1, 25), padding=(0, 12))
    self.conv1_4 = nn.Conv2d(1, 10, ( 1, 11), padding=(0,  5))
    self.bnc1 = nn.BatchNorm2d(40)
    self.conv2 = nn.Conv2d(40, 40, (22, 1))
    self.bnc2 = nn.BatchNorm2d(40)
    self.fc = nn.Linear(30 * 40, 4)
  
  def forward(self, x):
    xold = x
    c1 = self.conv1_1(x)
    c2 = self.conv1_2(x)
    c3 = self.conv1_3(x)
    c4 = self.conv1_4(x)
    x = torch.cat((c1, c2, c3, c4), 1)
    x = self.bnc1(F.elu(x))
    x = F.elu(self.bnc2(self.conv2(x)))
    x = F.dropout2d(x, p=0.5)
    x = F.max_pool2d(x.permute(0, 2, 1, 3), (1, 75), stride=(1, 15))
    x = x.view(-1, 30 * 40)
    x = self.fc(x)
    return x

####CNN15

CNN12 with Google Inception Input

In [0]:
# N x 1 x 22 x 513
class CNN15(nn.Module):
  def __init__(self, time_batch=400):
    super(CNN15, self).__init__()
    self.conv1_1 = nn.Conv2d(1,  6, ( 1, 75), padding=(0, 37))
    self.conv1_2 = nn.Conv2d(1,  6, ( 1, 43), padding=(0, 21))
    self.conv1_3 = nn.Conv2d(1,  6, ( 1, 25), padding=(0, 12))
    self.conv1_4 = nn.Conv2d(1,  6, ( 1, 11), padding=(0,  5))
    self.bnc1 = nn.BatchNorm2d(24)
    self.conv2 = nn.Conv2d(24,  25, ( 22,  1))
    self.bnc2 = nn.BatchNorm2d(25)
    self.conv3 = nn.Conv2d( 1,  50, ( 25, 10))
    self.bnc3 = nn.BatchNorm2d(50)
    self.conv4 = nn.Conv2d( 1, 100, ( 50, 10))
    self.bnc4 = nn.BatchNorm2d(100)
    self.conv5 = nn.Conv2d( 1, 200, (100, 10))
    self.bnc5 = nn.BatchNorm2d(200)
    self.fc = nn.Linear(200 * 2, 4)
  
  def forward(self, x):
    c1 = self.conv1_1(x)
    c2 = self.conv1_2(x)
    c3 = self.conv1_3(x)
    c4 = self.conv1_4(x)
    x = torch.cat((c1, c2, c3, c4), 1)
    x = self.bnc1(F.elu(x))
    x = F.elu(self.bnc2(self.conv2(x)))
    x = F.dropout2d(x, p=0.5)
    x = F.max_pool2d(x.permute(0, 2, 1, 3), (1, 3))
    x = F.elu(self.bnc3(self.conv3(x)))
    x = F.dropout2d(x, p=0.5)
    x = F.max_pool2d(x.permute(0, 2, 1, 3), (1, 3))
    x = F.elu(self.bnc4(self.conv4(x)))
    x = F.dropout2d(x, p=0.5)
    x = F.max_pool2d(x.permute(0, 2, 1, 3), (1, 3))
    x = F.elu(self.bnc5(self.conv5(x)))
    x = F.dropout2d(x, p=0.5)
    x = F.max_pool2d(x.permute(0, 2, 1, 3), (1, 3))
    x = x.view(-1, 200 * 2)
    x = self.fc(x)
    return x

####CNN16

CNN13 with Google Inception Input

Negligible result

In [0]:
# N x 1 x 22 x 512
class CNN16(nn.Module):
  def __init__(self, time_batch=400):
    super(CNN16, self).__init__()
    self.conv1_1 = nn.Conv2d(1, 4, (22,  1))
    self.conv1_2 = nn.Conv2d(1, 4, (22,  5), padding=(0,  2))
    self.conv1_3 = nn.Conv2d(1, 4, (22, 11), padding=(0,  5))
    self.conv1_4 = nn.Conv2d(1, 4, (22, 25), padding=(0, 12))
    self.bnc1 = nn.BatchNorm2d(16)
    self.padding1 = nn.ZeroPad2d((16, 17, 0, 1))
    self.conv2 = nn.Conv2d(1, 4, (2, 32))
    self.bnc2 = nn.BatchNorm2d(4)
    self.padding2 = nn.ZeroPad2d(( 2,  1, 4, 3))
    self.conv3 = nn.Conv2d(4, 4, (8,  4))
    self.bnc3 = nn.BatchNorm2d(4)
    self.fc = nn.Linear(4 * 4 * 32, 4)
  
  def forward(self, x):
    c1 = self.conv1_1(x)
    c2 = self.conv1_2(x)
    c3 = self.conv1_3(x)
    c4 = self.conv1_4(x)
    x = torch.cat((c1, c2, c3, c4), 1)
    x = self.bnc1(F.elu(x))
    x = F.dropout(x, p=0.25)
    x = x.permute(0, 2, 1, 3)
    x = self.padding1(x)
    x = F.elu(self.conv2(x))
    x = self.bnc2(x)
    x = F.dropout(x, p=0.25)
    x = F.max_pool2d(x, (2, 4))
    x = self.padding2(x)
    x = F.elu(self.conv3(x))
    x = self.bnc2(x)
    x = F.dropout(x, p=0.25)
    x = F.max_pool2d(x, (2, 4))
    x = x.view(-1, 4 * 4 * 32)
    x = self.fc(x)
    return x

##Setup

In [0]:
data_dir = 'drive/ee239as/project_datasets'
#batch_size = 31
batch_size = 17
#batch_size = 4
time_batch = 513

In [33]:
EEGset = EEGMinimalContainer(data_dir)
#EEGset = EEGCroppedContainer(data_dir, time_batch)

train_loader = torch.utils.data.DataLoader(EEGset.train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = {}
for i in range(1, 10):
  test_loader[str(i)] = torch.utils.data.DataLoader(EEGset.test_dataset[str(i)],
                                                    batch_size=1,
                                                    shuffle=False)

EEGContainer X_train: (2108, 1, 22, 1000)
EEGContainer y_train: (2108,)
EEGContainer X_test1: (50, 1, 22, 1000)
EEGContainer y_test1: (50,)
EEGContainer X_test2: (50, 1, 22, 1000)
EEGContainer y_test2: (50,)
EEGContainer X_test3: (50, 1, 22, 1000)
EEGContainer y_test3: (50,)
EEGContainer X_test4: (50, 1, 22, 1000)
EEGContainer y_test4: (50,)
EEGContainer X_test5: (50, 1, 22, 1000)
EEGContainer y_test5: (50,)
EEGContainer X_test6: (50, 1, 22, 1000)
EEGContainer y_test6: (50,)
EEGContainer X_test7: (50, 1, 22, 1000)
EEGContainer y_test7: (50,)
EEGContainer X_test8: (50, 1, 22, 1000)
EEGContainer y_test8: (50,)
EEGContainer X_test9: (50, 1, 22, 1000)
EEGContainer y_test9: (50,)


In [0]:
num_epochs = 200
learning_rate = 1e-4

use_cuda = True

net = CNN15()
criterion = nn.CrossEntropyLoss()

ss = 50

In [0]:
if use_cuda and torch.cuda.is_available():
  net.cuda()

optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate, weight_decay=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=ss, gamma=0.1)

##Training

In [0]:
training_acc_arr = np.empty(num_epochs)
testing_acc_arr = np.empty((9, num_epochs))

for epoch in range(num_epochs):
  
  net.train()
  
  #scheduler.step()
  
  total = 0
  correct = 0
  
  for i, (signals, labels) in enumerate(train_loader):
    
    signals = signals.type(torch.FloatTensor)
    signals = Variable(signals)
    labels = labels.type(torch.LongTensor)
    labels_check = torch.squeeze(labels)
    labels = Variable(torch.squeeze(labels))
    loc = np.random.choice(1000-time_batch, 4, replace=False)
    s0 = signals[:, :, :, loc[0]:(loc[0]+time_batch)]
    s1 = signals[:, :, :, loc[1]:(loc[1]+time_batch)]
    s2 = signals[:, :, :, loc[2]:(loc[2]+time_batch)]
    s3 = signals[:, :, :, loc[3]:(loc[3]+time_batch)]
    signals = torch.cat((s0, s1, s2, s3), 0)
    labels_more = torch.cat([labels] * 4)
    if use_cuda and torch.cuda.is_available():
      signals = signals.cuda()
      labels_more = labels_more.cuda()
    optimizer.zero_grad()
    outputs = net(signals)
    loss = criterion(outputs, labels_more)
    loss.backward()
    optimizer.step()
    _, predicted = torch.max(outputs.data, 1)
    predicted = predicted.cpu()
    predicted = predicted.view(-1, 4).numpy()
    predicted, _ = stats.mode(predicted, axis=1)
    predicted = np.squeeze(predicted)
    total += labels.size(0)
    correct += (predicted == labels_check.numpy()).sum()
    
    if (i+1) % 31 == 0:
      print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Learning Rate: %.3e' 
            % (epoch+1, num_epochs, i+1, len(EEGset.train_dataset)//batch_size, 
               loss.data[0], scheduler.get_lr()[0]))
  
  # Training accuracy
  training_acc_arr[epoch] = (correct/total)
  print ('Training Accuracy: %.5f' % training_acc_arr[epoch])
  
  net.eval()
  
  # Testing accuracy
  for subject in range(9):
    total = 0
    correct = 0
    for signals, labels in test_loader[str(subject+1)]:
      signals = signals.type(torch.FloatTensor)
      signals = Variable(signals)
      labels = labels.type(torch.LongTensor)
      labels_check = torch.squeeze(labels)
      labels = Variable(torch.squeeze(labels))
      loc = np.random.choice(1000-time_batch, 4, replace=False)
      s0 = signals[:, :, :, loc[0]:(loc[0]+time_batch)]
      s1 = signals[:, :, :, loc[1]:(loc[1]+time_batch)]
      s2 = signals[:, :, :, loc[2]:(loc[2]+time_batch)]
      s3 = signals[:, :, :, loc[3]:(loc[3]+time_batch)]
      signals = torch.cat((s0, s1, s2, s3), 0)
      labels_more = torch.cat([labels] * 4)
      if use_cuda and torch.cuda.is_available():
        signals = signals.cuda()
        labels_more = labels_more.cuda()
      outputs = net(signals)
      _, predicted = torch.max(outputs.data, 1)
      predicted = predicted.cpu()
      predicted = predicted.view(-1, 4).numpy()
      predicted, _ = stats.mode(predicted, axis=1)
      predicted = np.squeeze(predicted)
      total += labels.size(0)
      correct += (predicted == labels_check.numpy()).sum()
    testing_acc_arr[subject, epoch] = (correct/total)
  print ('Testing Accuracy: ' + str(testing_acc_arr[:, epoch]))
  print ('Testing Accuracy Average: %.5f' % np.average(testing_acc_arr[:, epoch]))

In [0]:
'''
loss_arr = []
training_acc_arr = np.empty(num_epochs)
testing_acc_arr = np.empty((9, num_epochs))

j = 0
for epoch in range(num_epochs):
  
  net.train()
  
  #scheduler.step()
  
  for i, (signals, labels) in enumerate(train_loader):
    
    signals = signals.type(torch.FloatTensor)
    signals = Variable(signals)
    labels = labels.type(torch.LongTensor)
    labels = Variable(torch.squeeze(labels))
    
    if use_cuda and torch.cuda.is_available():
      signals = signals.cuda()
      labels = labels.cuda()
    
    optimizer.zero_grad()
    outputs = net(signals)
    
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    
    #loss_arr[j] = loss.data[0]
    #j += 1
    
    if (i+1) % 31 == 0:
      print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Learning Rate: %.3e' 
            % (epoch+1, num_epochs, i+1, len(EEGset.train_dataset)//batch_size, 
               loss.data[0], scheduler.get_lr()[0]))
  
  net.eval()
  
  # Training accuracy
  total = 0
  correct = 0
  for signals, labels in train_loader:
    signals = signals.type(torch.FloatTensor)
    signals = Variable(signals)
    labels = torch.squeeze(labels.type(torch.LongTensor))
    if use_cuda and torch.cuda.is_available():
      signals = signals.cuda()
      labels = labels.cuda()
    outputs = net(signals)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()
  training_acc_arr[epoch] = (correct/total)
  print ('Training Accuracy: %.5f' % training_acc_arr[epoch])
  
  
  # Testing accuracy
  for subject in range(9):
    total = 0
    correct = 0
    for signals, labels in test_loader[str(subject+1)]:
      signals = signals.type(torch.FloatTensor)
      signals = Variable(signals)
      labels = torch.squeeze(labels.type(torch.LongTensor))
      if use_cuda and torch.cuda.is_available():
        signals = signals.cuda()
        labels = labels.cuda()
      outputs = net(signals)
      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum()
    testing_acc_arr[subject, epoch] = (correct/total)
  print ('Testing Accuracy: ' + str(testing_acc_arr[:, epoch]))
  print ('Testing Accuracy Average: %.5f' % np.average(testing_acc_arr[:, epoch]))
  '''

###Print MATLAB Format

In [0]:
# Print Loss
#print('Loss = [', end='')
#for lossval in np.nditer(loss_arr):
#  print('%.5f, ' % lossval, end='')
#print('];')

# Print Training Accuracy
print('Training_Accuracy = [', end='')
for acc in np.nditer(training_acc_arr):
  print('%.5f, ' % acc, end='')
print('];')

# Print Testing Accuracy
print('Testing_Accuracy = [', end='')
for subject in range(9):
  for acc in np.nditer(testing_acc_arr[subject, :]):
    print('%.5f, ' % acc, end='')
  print('; ', end='')
print('];')